# Performance comparisons

In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [28]:
from weighted_concordance import *
from score_functions import wibs

In [3]:
# # importing module
# import sys

# # appending a path
# sys.path.append('C:/Users/thseale/Documents/Python Packages')

# from lifelines import CoxPHFitter
# from lifelines.utils import datetimes_to_durations, concordance_index

In [4]:
from lifelines import CoxPHFitter
from lifelines.utils import datetimes_to_durations, concordance_index

## Loading models

In [31]:
unweighted_cox_pca = pickle.load(open('unweighted_cox_pca.pkl', 'rb'))

In [32]:
unweighted_cox = pickle.load(open('unweighted_cox.pkl', 'rb'))

In [33]:
barlow_cox_pca = pickle.load(open("barlow_cox_pca.pkl", "rb"))

In [34]:
barlow_cox = pickle.load(open("barlow_cox.pkl", "rb"))

In [35]:
prentice_cox_pca = pickle.load(open("prentice_cox_pca.pkl", "rb"))

In [36]:
prentice_cox = pickle.load(open("prentice_cox.pkl", "rb"))

In [37]:
selfprentice_cox_pca = pickle.load(open("selfprentice_cox_pca.pkl", "rb"))

In [38]:
selfprentice_cox = pickle.load(open("selfprentice_cox.pkl", "rb"))

In [39]:
adjusted_cox_pca = pickle.load(open("adjusted_cox_pca.pkl", "rb"))

In [40]:
adjusted_cox = pickle.load(open("adjusted_cox.pkl", "rb"))

In [41]:
unweighted_cox_penalised = pickle.load(open("unweighted_cox_penalised.pkl", "rb"))

In [42]:
penalised_barlow_cox = pickle.load(open("penalised_barlow_cox.pkl", "rb"))

In [43]:
penalised_prentice_cox = pickle.load(open("penalised_prentice_cox.pkl", "rb"))

In [44]:
penalised_selfprentice_cox = pickle.load(open("penalised_selfprentice_cox.pkl", "rb"))

In [45]:
penalised_adjusted_cox = pickle.load(open("penalised_adjusted_cox.pkl", "rb"))

## Loading data

### Test data 

In [6]:
X_tr_test = pd.read_csv("D:/compiled_data/X_tr_test.csv").set_index("csid")

y_test = pd.read_csv("D:/compiled_data/y_test.csv").set_index("csid")

In [7]:
len(y_test)

213

In [8]:
def date_time_conversion(date):
    return(datetime.strptime(date,"%Y-%m-%dT%H:%M:%SZ"))

# Coverting the times into datetime format
y_tr_test = y_test.assign(
    event = y_test.ep_CKB0020_combined_ep,
    start_time = y_test.study_date.map(date_time_conversion),
    event_time = y_test.ep_CKB0020_combined_datedeveloped.map(date_time_conversion)
).drop(columns = y_test.columns)
y_tr_test["duration"] = datetimes_to_durations(y_tr_test["start_time"],y_tr_test["event_time"])[0]
y_tr_test["subcohort"] = y_test["subcohort"]
y_tr_test = y_tr_test.drop(columns = ["start_time","event_time"])
y_tr_test

,event,duration,subcohort
csid,,,
2022-00229-3793895,1,3100.0,False
2022-00229-2146926,1,1229.0,False
2022-00229-5726795,1,1415.0,False
2022-00229-2081726,1,1548.0,False
2022-00229-1019474,1,277.0,False
...,...,...,...
2022-00229-4639290,0,4140.0,True
2022-00229-3940363,0,4407.0,True
2022-00229-1634360,0,4866.0,True


### Training
For estimating censoring distributino for integrated brier score.

In [9]:
y_train = pd.read_csv("D:/compiled_data/y.csv").set_index("csid")

In [10]:
# Coverting the times into datetime format
y_tr_train = y_train.assign(
    event = y_train.ep_CKB0020_combined_ep,
    start_time = y_train.study_date.map(date_time_conversion),
    event_time = y_train.ep_CKB0020_combined_datedeveloped.map(date_time_conversion)
).drop(columns = y_train.columns)
y_tr_train["duration"] = datetimes_to_durations(y_tr_train["start_time"],y_tr_train["event_time"])[0]
y_tr_train["subcohort"] = y_train["subcohort"]
y_tr_train = y_tr_train.drop(columns = ["start_time","event_time"])
y_tr_train

,event,duration,subcohort
csid,,,
2022-00229-1799093,1,1192.0,False
2022-00229-4744170,1,1690.0,False
2022-00229-1082517,1,2035.0,False
2022-00229-3397875,1,2530.0,False
2022-00229-4372175,1,2216.0,False
...,...,...,...
2022-00229-5565114,0,4495.0,True
2022-00229-4902412,0,3448.0,True
2022-00229-4823882,0,895.0,True


## Predictions

In [46]:
event_times = y_tr_test["duration"]
event_observed = y_tr_test["event"]

### Unweighted Cox PH

In [47]:
unweighted_cox_test_preds = unweighted_cox.predict_partial_hazard(unweighted_cox_pca.transform(X_tr_test))

### Cox PH, Barlow Weights

In [48]:
barlow_cox_test_preds = barlow_cox.predict_partial_hazard(barlow_cox_pca.transform(X_tr_test))

### Cox PH, Prentice Weights

In [49]:
prentice_cox_test_preds = prentice_cox.predict_partial_hazard(prentice_cox_pca.transform(X_tr_test))

### Cox PH, Self-Prentice Weights

In [50]:
selfprentice_cox_test_preds = selfprentice_cox.predict_partial_hazard(selfprentice_cox_pca.transform(X_tr_test))

### Cox PH, Adjusted Weights

In [51]:
adjusted_cox_test_preds = adjusted_cox.predict_partial_hazard(adjusted_cox_pca.transform(X_tr_test))

### Penalised unweighted Cox PH

In [52]:
unweighted_cox_penalised_test_preds = unweighted_cox_penalised.predict_partial_hazard(X_tr_test)

### Penalised Cox PH, Barlow Weights

In [53]:
penalised_barlow_cox_test_preds = penalised_barlow_cox.predict_partial_hazard(X_tr_test)

### Penalised Cox PH, Prentice Weights

In [54]:
penalised_prentice_cox_test_preds = penalised_prentice_cox.predict_partial_hazard(X_tr_test)

### Penalised Cox PH, Prentice Weights

In [55]:
penalised_selfprentice_cox_test_preds = penalised_selfprentice_cox.predict_partial_hazard(X_tr_test)

### Penalised Cox PH, Adjusted Weights

In [56]:
penalised_adjusted_cox_test_preds = penalised_adjusted_cox.predict_partial_hazard(X_tr_test)

## Results

In [20]:
sample_fraction = len(y_tr_test)/(512724-1200)
sample_fraction

0.00041640274943111175

In [73]:
model_results = pd.DataFrame({
    "model": ["Unweighted Cox PH",
             "Cox PH, Barlow Weights",
             "Cox PH, Prentice Weights",
             "Cox PH, Self-Prentice Weights",
             "Cox PH, Adjusted Weights",
             "Penalized unweighted Cox PH",
             "Penalized Cox PH, Barlow Weights",
             "Penalized Cox PH, Prentice Weights",
             "Penalized Cox PH, Self-Prentice Weights",
             "Penalized Cox PH, Adjusted Weights"],
    "weighted_concordance": [weighted_concordance(event_times,unweighted_cox_test_preds, event_observed,sample_fraction),
                            weighted_concordance(event_times,barlow_cox_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,prentice_cox_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,selfprentice_cox_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,adjusted_cox_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,unweighted_cox_penalised_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,penalised_barlow_cox_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,penalised_prentice_cox_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,penalised_selfprentice_cox_test_preds, event_observed,sample_fraction),
                             weighted_concordance(event_times,penalised_adjusted_cox_test_preds, event_observed,sample_fraction)],
    "concordance": [concordance_index(event_times,-unweighted_cox_test_preds, event_observed),
                            concordance_index(event_times,-barlow_cox_test_preds, event_observed),
                             concordance_index(event_times,-prentice_cox_test_preds, event_observed),
                             concordance_index(event_times,-selfprentice_cox_test_preds, event_observed),
                             concordance_index(event_times,-adjusted_cox_test_preds, event_observed),
                             concordance_index(event_times,-unweighted_cox_penalised_test_preds, event_observed),
                             concordance_index(event_times,-penalised_barlow_cox_test_preds, event_observed),
                             concordance_index(event_times,-penalised_prentice_cox_test_preds, event_observed),
                             concordance_index(event_times,-penalised_selfprentice_cox_test_preds, event_observed),
                             concordance_index(event_times,-penalised_adjusted_cox_test_preds, event_observed)],
    "weighted_integrated_brier_scores": [wibs(unweighted_cox, unweighted_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(barlow_cox, barlow_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(prentice_cox, prentice_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(selfprentice_cox, selfprentice_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(adjusted_cox, adjusted_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(unweighted_cox_penalised, X_tr_test, y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(penalised_barlow_cox, X_tr_test, y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(penalised_prentice_cox, X_tr_test, y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(penalised_selfprentice_cox, X_tr_test, y_tr_test, y_tr_train, 1/sample_fraction),
                                         wibs(penalised_adjusted_cox, X_tr_test, y_tr_test, y_tr_train, 1/sample_fraction)],
    "integrated_brier_scores": [wibs(unweighted_cox, unweighted_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1),
                                         wibs(barlow_cox, barlow_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1),
                                         wibs(prentice_cox, prentice_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1),
                                         wibs(selfprentice_cox, selfprentice_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1),
                                         wibs(adjusted_cox, adjusted_cox_pca.transform(X_tr_test), y_tr_test, y_tr_train, 1),
                                         wibs(unweighted_cox_penalised, X_tr_test, y_tr_test, y_tr_train, 1),
                                         wibs(penalised_barlow_cox, X_tr_test, y_tr_test, y_tr_train, 1),
                                         wibs(penalised_prentice_cox, X_tr_test, y_tr_test, y_tr_train, 1),
                                         wibs(penalised_selfprentice_cox, X_tr_test, y_tr_test, y_tr_train, 1),
                                         wibs(penalised_adjusted_cox, X_tr_test, y_tr_test, y_tr_train, 1)],
    
})
model_results

,model,weighted_concordance,concordance,weighted_integrated_brier_scores,integrated_brier_scores
0,Unweighted Cox PH,0.681771,0.625103,157.291154,0.162683
1,"Cox PH, Barlow Weights",0.671967,0.616854,0.306693,0.305061
2,"Cox PH, Prentice Weights",0.704533,0.659369,126.954077,0.163855
3,"Cox PH, Self-Prentice Weights",0.507916,0.519386,1.830794,0.305627
4,"Cox PH, Adjusted Weights",0.682983,0.641157,101.808395,0.168252
5,Penalized unweighted Cox PH,0.698799,0.655879,1252.810981,0.524371
6,"Penalized Cox PH, Barlow Weights",0.720715,0.666984,127.554059,0.155176
7,"Penalized Cox PH, Prentice Weights",0.724692,0.671299,212.548105,0.158433
8,"Penalized Cox PH, Self-Prentice Weights",0.513270,0.507900,0.317061,0.303849
9,"Penalized Cox PH, Adjusted Weights",0.724785,0.671299,233.995957,0.161597
